In [1]:
1+1

2

In [2]:
import os

In [3]:
%pwd

'e:\\MLOPS\\TextSummarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\MLOPS\\TextSummarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int



In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config


In [9]:
 from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
 from transformers import TrainingArguments, Trainer
 from transformers import DataCollatorForSeq2Seq
 import torch
 from datasets import load_from_disk


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model_pegasus
        )

        # loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # training arguments
        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )

        # trainer
        trainer = Trainer(
            model=model_pegasus,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()
        # save the model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.json(self.config.root_dir,"tokenizer"))

In [ ]:
config=ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-09-10 10:45:53,280: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-10 10:45:53,303: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-10 10:45:53,310: INFO: common]: Created directory at: artifacts
[2025-09-10 10:45:53,310: INFO: common]: Created directory at: artifacts/model_trainer


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\shiva\AppData\Local\Temp\ipykernel_372\4292523788.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
e:\MLOPS\TextSummarizer\.venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memor